# Script de Transformación.  
Se realiza el proceso de transformación para cada una de las tablas, así como la codificación del Fact para optimización de espacio. Por último se añade la información a la base de datos 

In [1]:
# En caso de no estar activo se activa el entorno virtual 
!.\venv\Scripts\activate

In [2]:
# Se realiza el cargue de las librerías necesarias 
import pandas as pd #Transformacion de datos 
import os # Estandarización de urls para correcta ejecución en cualquier dispositivo
import duckdb

## Cargue de la base de datos cruda
Se realiza el cargue de la información cruda y se eliminan aquellas columnas que no contienen nada de información

In [3]:
# Se identifica ubicación del repositorio
user = os.getcwd()
path_data = rf"{user}\00 Inputs\netflix_titles.csv"

# Se realiza el cargue de la información
df = pd.read_csv(path_data, encoding = 'latin1')

# Se eliminan columnas que no contiene nada de informacion
df_cleaned = df.dropna(axis=1, how='all')

# Proceso de transformación de acuerdo al modelo entidad relación 

A continuación se realizará el proceso de estructuración de cada una de las tablas de **Dimensión** con el fin de realizar el cargue en la base de datos

### Dimension de fechas

In [4]:
# Se convierte la columna date_added to datetime
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

# Obtener la fecha mínima y máxima
min_date = df['date_added'].min()
max_date = df['date_added'].max()
print("min", min_date, "max", max_date)

# Crear un rango de fechas desde min_date hasta max_date
date_range = pd.date_range(start=min_date, end=max_date)

# Crear el DataFrame DimDate con el rango de fechas
DimDate = pd.DataFrame(date_range, columns=['date'])

# Extraer día, mes y año de la columna 'date'
DimDate['day'] = DimDate['date'].dt.day
DimDate['month'] = DimDate['date'].dt.strftime('%B')  # Nombre completo del mes
DimDate['year'] = DimDate['date'].dt.year

# Agregar un ID único para cada fecha
DimDate = DimDate.reset_index().rename(columns={'index': 'date_id'})
DimDate = DimDate[['date', 'day', 'month', 'year']]
DimDate

min 2008-01-01 00:00:00 max 2024-04-05 00:00:00


,date,day,month,year
0,2008-01-01,1,January,2008
1,2008-01-02,2,January,2008
2,2008-01-03,3,January,2008
3,2008-01-04,4,January,2008
4,2008-01-05,5,January,2008
...,...,...,...,...
5935,2024-04-01,1,April,2024
5936,2024-04-02,2,April,2024
5937,2024-04-03,3,April,2024
5938,2024-04-04,4,April,2024


### Dimension de Tipo de Show

In [5]:
# Dimension de Tipo de Show. 
DimShowType = df_cleaned[['type']].drop_duplicates().reset_index(drop=True)

# Convertir el índice en una columna llamada 'type_id'
DimShowType['type_id'] = DimShowType.index

# Reorganizar las columnas para que 'type_id' sea la primera
DimShowType = DimShowType[['type_id', 'type']]

DimShowType.head()

,type_id,type
0,0,Movie
1,1,TV Show


### Dimension Paises donde se ha reproducido

In [6]:
# Dimension de Tipo de Ciudades. 
# Crear DimCountries utilizando chaining
DimCountries = (
    df_cleaned[['country']]
    .dropna()
    .assign(country=lambda df: df['country'].str.split(','))
    .explode('country')
    .assign(country=lambda df: df['country'].str.strip())
    .loc[lambda df: df['country'] != '']  # Filtro para eliminar valores vacíos
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={'index': 'country_id'})
)

# Mostrar las primeras filas del DataFrame limpio para ver los cambios
DimCountries.head()

,country_id,country
0,0,United States
1,1,South Africa
2,2,India
3,3,Ghana
4,4,Burkina Faso


### Dimension de Elenco 
Se incluyen aquellas personas que participaron en la ejecución de la película o serie, con su clasificación de actor o director. 

In [26]:
import pandas as pd

# Crear DimCast_actors
DimCast_actors = (
    df_cleaned[['cast']]
    .dropna()
    .assign(cast=lambda df: df['cast'].str.split(','))
    .explode('cast')
    .assign(cast=lambda df: df['cast'].str.strip(), 
            type="actor")
    .drop_duplicates()
    .reset_index(drop=True)
)

# Crear DimCast_directors
DimCast_directors = (
    df_cleaned[['director']]
    .dropna()
    .assign(director=lambda df: df['director'].str.split(','))
    .explode('director')
    .assign(director=lambda df: df['director'].str.strip(), 
            type="director")
    .drop_duplicates()
    .reset_index(drop=True)
    .rename(columns={'director': 'cast'})
)

# Combinar ambos DataFrames
DimCast = pd.concat([DimCast_actors, DimCast_directors], ignore_index=True).drop_duplicates().reset_index(drop=True)

# Asignar un cast_id único
DimCast = DimCast.reset_index().rename(columns={'index': 'cast_id'})

# Verificar el resultado
DimCast.head()


,cast_id,cast,type
0,0,Ama Qamata,actor
1,1,Khosi Ngema,actor
2,2,Gail Mabalane,actor
3,3,Thabang Molaba,actor
4,4,Dillon Windvogel,actor


### Dimensión de Género de la Película

In [8]:
DimGenres = (
    df_cleaned[['listed_in']]
    .dropna()
    .assign(listed_in=lambda df: df['listed_in'].str.split(','))
    .explode('listed_in')
    .assign(listed_in=lambda df: df['listed_in'].str.strip())
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={'index': 'genere_id', 'listed_in': 'name'})
)

# Mostrar las primeras filas del DataFrame limpio para ver los cambios
DimGenres.head()

,genere_id,name
0,0,Documentaries
1,1,International TV Shows
2,2,TV Dramas
3,3,TV Mysteries
4,4,Crime TV Shows


### Proceso de transformación de la tabla de Hecho 
En este proceso se tomo la base de datos original, se reestructuró para que fuera de forma tabular y por último se codificó de acuerdo a las dimensiones creadas anteriormente, esto para reducir el peso de la tabla ya que el tipo de dato string consume más bytes que el tipo integer. 

In [11]:
# Proceso de Transformación del FactShows
# Crear la tabla de hechos FactShows
FactShows = df_cleaned.copy()

FactShows['cast_and_director'] = FactShows[['director', 'cast']].apply(lambda x: ', '.join(x.dropna()), axis=1)

# Separar los valores de las columnas 'country', 'director' y 'listed_in'
FactShows = (
    FactShows[['show_id', 'type', 'title', 'cast_and_director', 'country', 'release_year', 'rating', 'duration', 'listed_in', 'description', 'date_added']]
    .assign(country=lambda df: df['country'].str.split(','))
    .explode('country')
    .assign(country=lambda df: df['country'].str.strip())
    .loc[lambda df: df['country'] != '']
    .assign(cast_and_director=lambda df: df['cast_and_director'].str.split(','))
    .explode('cast_and_director')
    .assign(cast_and_director=lambda df: df['cast_and_director'].str.strip())
    .assign(listed_in=lambda df: df['listed_in'].str.split(','))
    .explode('listed_in')
    .assign(listed_in=lambda df: df['listed_in'].str.strip())
)

# Renombrar la columna 'listed_in' a 'genre' para mayor claridad
FactShows = FactShows.rename(columns={'listed_in': 'genre'})

# Crear diccionarios para codificar las dimensiones
show_type_dict = DimShowType.set_index('type')['type_id'].to_dict()
country_dict = DimCountries.set_index('country')['country_id'].to_dict()
cast_dict = DimCast.set_index('elenco')['cast_id'].to_dict()
genre_dict = DimGenres.set_index('name')['genere_id'].to_dict()

# Codificar las columnas en FactShows
# Asignar las nuevas columnas con los IDs correspondientes y manejar valores no mapeados
FactShows['type_id'] = FactShows['type'].map(show_type_dict).astype('Int64')
FactShows['country_id'] = FactShows['country'].map(country_dict).astype('Int64')
FactShows['cast_id'] = FactShows['cast_and_director'].map(cast_dict).astype('Int64')
FactShows['genre_id'] = FactShows['genre'].map(genre_dict).astype('Int64')

FactShows['date_added'] = pd.to_datetime(FactShows['date_added'], errors='coerce')

# Formatear la fecha al formato YYYY-MM-DD
FactShows['date_added'] = FactShows['date_added'].dt.strftime('%Y-%m-%d')

# Verificar los tipos de datos de las nuevas columnas
print(FactShows.dtypes)
# Seleccionar y reorganizar las columnas para la tabla de hechos
FactShows = FactShows[['show_id', 'type_id', 'title', 'cast_id', 'country_id', 'date_added', 'release_year', 'rating', 'duration', 'genre_id', 'description']]

# Mostrar las primeras filas del DataFrame FactShows codificado
FactShows.head()

show_id              object
type                 object
title                object
cast_and_director    object
country              object
release_year          int64
rating               object
duration             object
genre                object
description          object
date_added           object
type_id               Int64
country_id            Int64
cast_id               Int64
genre_id              Int64
dtype: object


,show_id,type_id,title,cast_id,country_id,date_added,release_year,rating,duration,genre_id,description
0,s1,0,Dick Johnson Is Dead,0,0,2021-09-25,2020,PG-13,90 min,0,"As her father nears the end of his life, filmm..."
1,s2,1,Blood & Water,0,1,2021-09-24,2021,TV-MA,2 Seasons,1,"After crossing paths at a party, a Cape Town t..."
1,s2,1,Blood & Water,0,1,2021-09-24,2021,TV-MA,2 Seasons,2,"After crossing paths at a party, a Cape Town t..."
1,s2,1,Blood & Water,0,1,2021-09-24,2021,TV-MA,2 Seasons,3,"After crossing paths at a party, a Cape Town t..."
1,s2,1,Blood & Water,1,1,2021-09-24,2021,TV-MA,2 Seasons,1,"After crossing paths at a party, a Cape Town t..."


## Cargue de cada una de las tablas a la base de datos 

In [12]:
con = duckdb.connect('my_database.db')

### Se inserta la información en la base de datos 

#### FactShows

In [31]:
# Crear la tabla FactShows
con.register('temp_factshows', FactShows)
con.execute('INSERT INTO FactShows SELECT * FROM temp_factshows')
# Eliminar la tabla temporal
con.execute('DROP VIEW IF EXISTS temp_factshows')
print("Se insertaron registros", con.execute('SELECT COUNT(*) FROM FactShows').fetchall(), "a la tabla FactShows")

#### DimDate

In [21]:
# Crear la tabla DimDate y poblarla usando to_sql
con.register('temp_dimdate', DimDate)
con.execute('INSERT INTO DimDate SELECT * FROM temp_dimdate')
# Eliminar la tabla temporal
con.execute('DROP VIEW IF EXISTS temp_dimdate')
print("Se insertaron registros", con.execute('SELECT COUNT(*) FROM DimDate').fetchall(), "a la tabla DimDate")

Se insertaron registros [(5940,)] a la tabla DimDate


### DimCast

In [27]:
# Crear la tabla DimCast y poblarla usando to_sql
con.register('temp_dimcast', DimCast)
con.execute('INSERT INTO DimCast SELECT * FROM temp_dimcast')
# Eliminar la tabla temporal
con.execute('DROP VIEW IF EXISTS temp_dimcast')
print("Se insertaron registros", con.execute('SELECT COUNT(*) FROM DimCast').fetchall(), "a la tabla DimCast")

Se insertaron registros [(41460,)] a la tabla DimCast


#### DimShowType

In [33]:
# Crear la tabla DimShowType y poblarla usando to_sql
con.register('temp_dimshowtype', DimShowType)
con.execute('INSERT INTO DimShowType SELECT * FROM temp_dimshowtype')
# Eliminar la tabla temporal
con.execute('DROP VIEW IF EXISTS temp_dimshowtype')
print("Se insertaron registros", con.execute('SELECT COUNT(*) FROM DimShowType').fetchall(), "a la tabla DimShowType")

Se insertaron registros [(2,)] a la tabla DimShowType


#### DimGenres

In [34]:
# Crear la tabla DimGenres y poblarla usando to_sql
con.register('temp_dimgenre', DimGenres)
con.execute('INSERT INTO DimGenres SELECT * FROM temp_dimgenre')
# Eliminar la tabla temporal
con.execute('DROP VIEW IF EXISTS temp_dimgenre')
print("Se insertaron registros", con.execute('SELECT COUNT(*) FROM DimGenres').fetchall(), "a la tabla DimGenres")

Se insertaron registros [(48,)] a la tabla DimGenres


#### DimCounties

In [35]:
# Crear la tabla DimCountries y poblarla usando to_sql
con.register('temp_dimcountry', DimCountries)
con.execute('INSERT INTO DimCountries SELECT * FROM temp_dimcountry')
# Eliminar la tabla temporal
con.execute('DROP VIEW IF EXISTS temp_dimcountry')
print("Se insertaron registros", con.execute('SELECT COUNT(*) FROM DimCountries').fetchall(), "a la tabla DimCountries")

Se insertaron registros [(122,)] a la tabla DimCountries


In [36]:
# Se cierra la conexión a la base de datos
con.close()